## Responses APIを使用した推論モデルのパフォーマンス向上

### 概要

OpenAIの最新推論モデルでResponses APIを活用することで、アプリケーションにおいてより高い知能、低コスト、そしてより効率的なトークン使用を実現できます。このAPIは推論要約へのアクセスも可能にし、ホスト型ツール使用などの機能をサポートし、さらなる柔軟性とパフォーマンスの向上を目指した今後の機能強化に対応するよう設計されています。

最近、私たちは推論能力とエージェント的なツール使用を組み合わせることに優れた、最先端の推論モデル2つ、o3とo4-miniをリリースしました。多くの方が知らないのは、私たちの（比較的）新しいResponses APIを最大限に活用することで、これらのモデルの性能を向上させることができるということです。このクックブックでは、これらのモデルから最大限の効果を得る方法を示し、推論と関数呼び出しが舞台裏でどのように動作するかを探ります。モデルに以前の推論項目へのアクセスを与えることで、最大の知能と最低のコストで動作することを保証できます。

私たちは、別の[クックブック](https://cookbook.openai.com/examples/responses_api/responses_example)と[APIリファレンス](https://platform.openai.com/docs/api-reference/responses)でResponses APIを紹介しました。主なポイント：Responses APIはCompletions APIに似ていますが、改善と追加機能があります。また、Responsesの暗号化されたコンテンツも展開しており、ステートフルな方法でAPIを使用できない方にとってさらに便利になりました！

## 推論モデルの仕組み

Responses APIがどのように役立つかを詳しく説明する前に、[推論モデル](https://platform.openai.com/docs/guides/reasoning?api-mode=responses)の仕組みを簡単に確認しましょう。o3やo4-miniのようなモデルは、問題を段階的に分解し、その推論を符号化した内部的な思考の連鎖を生成します。安全性のため、これらの推論トークンはユーザーには要約された形でのみ公開されます。

マルチステップの会話では、推論トークンは各ターンの後に破棄され、各ステップからの入力トークンと出力トークンが次のステップに送られます。

![reasoning-context](../../images/reasoning-turns.png)
図は私たちの[ドキュメント](https://platform.openai.com/docs/guides/reasoning?api-mode=responses#how-reasoning-works)から借用

返されるレスポンスオブジェクトを確認してみましょう：

In [3]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
response = client.responses.create(
    model="o4-mini",
    input="tell me a joke",
)


In [9]:
import json

print(json.dumps(response.model_dump(), indent=2))


{
  "id": "resp_6820f382ee1c8191bc096bee70894d040ac5ba57aafcbac7",
  "created_at": 1746989954.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "o4-mini-2025-04-16",
  "object": "response",
  "output": [
    {
      "id": "rs_6820f383d7c08191846711c5df8233bc0ac5ba57aafcbac7",
      "summary": [],
      "type": "reasoning",
      "status": null
    },
    {
      "id": "msg_6820f3854688819187769ff582b170a60ac5ba57aafcbac7",
      "content": [
        {
          "annotations": [],
          "text": "Why don\u2019t scientists trust atoms?  \nBecause they make up everything!",
          "type": "output_text"
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "max_output_tokens": null,
  "previous_response_id": null,
  "reasoning": {
    "effort": "medium",
    "generate

レスポンスオブジェクトのJSONダンプから、モデルが`output_text`に加えて推論アイテムも生成していることがわかります。このアイテムはモデルの内部推論トークンを表し、IDとして公開されます。例えば、ここでは`rs_6820f383d7c08191846711c5df8233bc0ac5ba57aafcbac7`です。Responses APIはステートフルであるため、これらの推論トークンは永続化されます。後続のメッセージにそれらのIDを含めるだけで、将来のレスポンスが同じ推論アイテムにアクセスできるようになります。マルチターン会話で`previous_response_id`を使用する場合、モデルは以前に生成されたすべての推論アイテムに自動的にアクセスできます。

また、モデルが生成した推論トークンの数も確認できます。例えば、10個の入力トークンに対して、レスポンスには148個の出力トークンが含まれており、そのうち128個は最終的なアシスタントメッセージには表示されない推論トークンです。

待って—図では前のターンからの推論が破棄されると示されていませんでしたか？それなら、なぜ後のターンでそれを渡し返す必要があるのでしょうか？

素晴らしい質問です！典型的なマルチターン会話では、推論アイテムやトークンを含める必要はありません—モデルはそれらなしで最適な出力を生成するように訓練されています。しかし、ツール使用が関わる場合は状況が変わります。ターンに関数呼び出しが含まれる場合（API外部での追加のラウンドトリップが必要になる可能性があります）、推論アイテムを含める必要があります—`previous_response_id`を使用するか、明示的に推論アイテムを`input`に追加することで。簡単な関数呼び出しの例でこれがどのように機能するかを見てみましょう。

In [14]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

context = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="o4-mini",
    input=context,
    tools=tools,
)


response.output

[ResponseReasoningItem(id='rs_68210c71a95c81919cc44afadb9d220400c77cc15fd2f785', summary=[], type='reasoning', status=None),
 ResponseFunctionToolCall(arguments='{"latitude":48.8566,"longitude":2.3522}', call_id='call_9ylqPOZUyFEwhxvBwgpNDqPT', name='get_weather', type='function_call', id='fc_68210c78357c8191977197499d5de6ca00c77cc15fd2f785', status='completed')]

推論を行った後、o4-miniモデルはより多くの情報が必要であると判断し、それを取得するために関数を呼び出します。私たちは関数を呼び出し、その出力をモデルに返すことができます。重要なのは、モデルの知能を最大化するために、次のターンのコンテキストにすべての出力を追加することで、推論項目を含めるべきだということです。

In [15]:
context += response.output # Add the response to the context (including the reasoning item)

tool_call = response.output[1]
args = json.loads(tool_call.arguments)


# calling the function
result = get_weather(args["latitude"], args["longitude"]) 

context.append({                               
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

# we are calling the api again with the added function call output. Note that while this is another API call, we consider this as a single turn in the conversation.
response_2 = client.responses.create(
    model="o4-mini",
    input=context,
    tools=tools,
)

print(response_2.output_text)

The current temperature in Paris is 16.3°C. If you’d like more details—like humidity, wind speed, or a brief description of the sky—just let me know!


この簡単な例では利点が明確に示されていないかもしれません。モデルは推論項目の有無に関わらず良好に動作する可能性が高いためです。しかし、私たちの独自のテストでは異なる結果が得られました。SWE-benchのようなより厳密なベンチマークでは、推論項目を含めることで、同じプロンプトと設定において約**3%の改善**が見られました。

## キャッシュ

上記で示したように、推論モデルは推論トークンと完了トークンの両方を生成し、APIはこれらを異なって処理します。この違いはキャッシュの動作に影響し、パフォーマンスと遅延の両方に影響を与えます。以下の図はこれらの概念を説明しています：

![reasoning-context](../../images/responses-diagram.png)

ターン2では、ターン1からの推論アイテムは無視され削除されます。これは、モデルが前のターンからの推論アイテムを再利用しないためです。その結果、図の4番目のAPI呼び出しでは、これらの推論アイテムがプロンプトから欠落しているため、完全なキャッシュヒットを実現できません。ただし、これらを含めることは無害です—APIは現在のターンに関連しない推論アイテムを単純に破棄します。キャッシュは1024トークンより長いプロンプトにのみ影響することを覚えておいてください。私たちのテストでは、Completions APIからResponses APIに切り替えることで、キャッシュ利用率が40%から80%に向上しました。キャッシュ利用率の向上により、コストの削減（例えば、`o4-mini`のキャッシュされた入力トークンは、キャッシュされていないものより75%安価）とレイテンシの改善が実現されます。

## 暗号化推論アイテム

[ゼロデータ保持（ZDR）](https://openai.com/enterprise-privacy/)要件を持つ組織など、一部の組織では、コンプライアンスやデータ保持ポリシーのため、Responses APIをステートフルな方法で使用できません。このようなケースをサポートするため、OpenAIは[暗号化推論アイテム](https://platform.openai.com/docs/guides/reasoning?api-mode=responses#encrypted-reasoning-items)を提供しており、ワークフローをステートレスに保ちながら推論アイテムの恩恵を受けることができます。

暗号化推論アイテムを使用するには：
- API呼び出しの`include`フィールドに`["reasoning.encrypted_content"]`を追加します。
- APIは推論トークンの暗号化版を返し、通常の推論アイテムと同様に将来のリクエストで渡すことができます。

ZDR組織の場合、OpenAIは自動的に`store=false`を強制します。リクエストに`encrypted_content`が含まれている場合、それはメモリ内で復号化され（ディスクに書き込まれることはありません）、次のレスポンス生成に使用された後、安全に破棄されます。新しい推論トークンは即座に暗号化されてあなたに返され、中間状態が永続化されることはありません。

これがどのように動作するかを示すコードの簡単な更新は以下の通りです：

In [39]:
context = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="o3",
    input=context,
    tools=tools,
    store=False, #store=false, just like how ZDR is enforced
    include=["reasoning.encrypted_content"] # Encrypted chain of thought is passed back in the response
)

In [34]:
# take a look at the encrypted reasoning item
print(response.output[0]) 

ResponseReasoningItem(id='rs_6821243503d481919e1b385c2a154d5103d2cbc5a14f3696', summary=[], type='reasoning', status=None, encrypted_content='gAAAAABoISQ24OyVRYbkYfukdJoqdzWT-3uiErKInHDC-lgAaXeky44N77j7aibc2elHISjAvX7OmUwMU1r7NgaiHSVWL5BtWgXVBp4BMFkWZpXpZY7ff5pdPFnW3VieuF2cSo8Ay7tJ4aThGUnXkNM5QJqk6_u5jwd-W9cTHjucw9ATGfGqD2qHrXyj6NEW9RmpWHV2SK41d5TpUYdN0xSuIUP98HBVZ2VGgD4MIocUm6Lx0xhRl9KUx19f7w4Sn7SCpKUQ0zwXze8UsQOVvv1HQxk_yDosbIg1SylEj38H-DNLil6yUFlWI4vGWcPn1bALXphTR2EwYVR52nD1rCFEORUd7prS99i18MUMSAhghIVv9OrpbjmfxJh8bSQaHu1ZDTMWcfC58H3i8KnogmI7V_h2TKAiLTgSQIkYRHnV3hz1XwaUqYAIhBvP6c5UxX-j_tpYpB_XCpD886L0XyJxCmfr9cwitipOhHr8zfLVwMI4ULu-P3ftw7QckIVzf71HFLNixrQlkdgTn-zM6aQl5BZcJgwwn3ylJ5ji4DQTS1H3AiTrFsEt4kyiBcE2d7tYA_m3G8L-e4-TuTDdJZtLaz-q8J12onFaKknGSyU6px8Ki4IPqnWIJw8SaFMJ5fSUYJO__myhp7lbbQwuOZHIQuvKutM-QUuR0cHus_HtfWtZZksqvVCVNBYViBxD2_KvKJvR-nN62zZ8sNiydIclt1yJfIMkiRErfRTzv92hQaUtdqz80UiW7FBcN2Lnzt8awXCz1pnGyWy_hNQe8C7W35zRxJDwFdb-f3VpanJT0tNmU5bfEWSXcIVmiMZL1clwzVNryf9Gk482LaWPwhVYrh

`include=["reasoning.encrypted_content"]`を設定することで、返されるreasoningアイテムに`encrypted_content`フィールドが表示されるようになります。この暗号化されたコンテンツは、モデルの推論状態を表しており、OpenAIがデータを保持することなく、完全にクライアント側で永続化されます。その後、以前にreasoningアイテムで行ったのと同様に、これを再び渡すことができます。

In [40]:
context += response.output # Add the response to the context (including the encrypted chain of thought)
tool_call = response.output[1]
args = json.loads(tool_call.arguments)



result = 20 #mocking the result of the function call

context.append({                               
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="o3",
    input=context,
    tools=tools,
    store=False,
    include=["reasoning.encrypted_content"]
)

print(response_2.output_text)

It’s currently about 20 °C in Paris.


`include`フィールドへの簡単な変更により、暗号化された推論アイテムを返して、モデルのインテリジェンス、コスト、レイテンシのパフォーマンス向上に活用できるようになりました。

これで、最新の推論モデルを最大限に活用するための知識が完全に身につきました！

## 推論要約

Responses APIのもう一つの有用な機能は、推論要約をサポートしていることです。生の思考連鎖トークンは公開していませんが、ユーザーはその[要約](https://platform.openai.com/docs/guides/reasoning?api-mode=responses#reasoning-summaries)にアクセスできます。

In [9]:
# Make a hard call to o3 with reasoning summary included

response = client.responses.create(
    model="o3",
    input="What are the main differences between photosynthesis and cellular respiration?",
    reasoning={"summary": "auto"},

    
)

# Extract the first reasoning summary text from the response object
first_reasoning_item = response.output[0]  # Should be a ResponseReasoningItem
first_summary_text = first_reasoning_item.summary[0].text if first_reasoning_item.summary else None
print("First reasoning summary text:\n", first_summary_text)



First reasoning summary text:
 **Analyzing biological processes**

I think the user is looking for a clear explanation of the differences between certain processes. I should create a side-by-side comparison that lists out key elements like the formulas, energy flow, locations, reactants, products, organisms involved, electron carriers, and whether the processes are anabolic or catabolic. This structured approach will help in delivering a comprehensive answer. It’s crucial to cover all aspects to ensure the user understands the distinctions clearly.


推論要約テキストを使用すると、ユーザーにモデルの思考プロセスを覗く窓を提供できます。例えば、複数の関数呼び出しを含む会話中に、ユーザーは最終的なアシスタントメッセージを待つことなく、どの関数が呼び出されたかと各呼び出しの背後にある推論の両方を確認できます。これにより、アプリケーションのユーザーエクスペリエンスに透明性とインタラクティブ性が追加されます。

## まとめ

OpenAI Responses APIと最新の推論モデルを活用することで、アプリケーションにおいてより高い知能、改善された透明性、そしてより大きな効率性を実現できます。推論サマリーを利用する場合でも、コンプライアンス向けの暗号化された推論アイテムを使用する場合でも、コストとレイテンシを最適化する場合でも、これらのツールはより堅牢でインタラクティブなAI体験の構築を可能にします。

楽しい開発を！